**(a) (4 points) Load the data file to your S3 bucket. Using the pandas library, read the csv
data file and create a data-frame called college.**

In [1]:
import boto3
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action= 'ignore', category=FutureWarning)

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import MinMaxScaler

#defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'craig-shaffer-data-445-bucket'
bucket = s3.Bucket(bucket_name)

#defining the file to be read from s3 bucket
file_key = 'College.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datafile
college = pd.read_csv(file_content_stream)
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


**(b) (3 points) Change the Private variable from a categorical variable to a numerical
variable. That is, change Yes to 1 and No to 0.**

In [2]:
#change yes/no to 1/0
college['Private'].replace(['Yes', 'No'], [1,0], inplace = True)
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,1,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,1,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,1,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


**(c) (5 points) Using Private, F.Undergrad, P.Undergrad, Outstate, Room.Board, Books, Personal, S.F.Ratio and Grad.Rate as the predictor variables, and Apps as the target variable, split the data into train (80%) and test (20%)**

In [18]:
#defining input and target variables
x = college[['Private', 'F.Undergrad', 'P.Undergrad', 'Outstate', 'Room.Board', 'Books', 'Personal', 'S.F.Ratio', 'Grad.Rate']]
y = college['Apps']

#split into train and test
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

**(d) (3 points) Using the MinMaxScaler, transform the input variables in the train and test
dataset to 0-1 scale.**

In [19]:
#transforming the input data
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

**(e) (5 points) Using the train dataset, build a linear regression model. After that, use this
model to predict on the test dataset. Report the MSE of this model.**


In [20]:
#building linear regression rodel
lm_md= LinearRegression().fit(x_train, y_train)

#predict on test
lm_pred= lm_md.predict(x_test)

#compute model mse
mse1 = np.mean(np.power((y_test-lm_pred),2))
print("The mse of the linear model is:",mse1)

The mse of the linear model is: 2708734.6387247397


**(f) (8 points) Using the train dataset, build a ridge regression model as follows:**
 - **First estimate the optimal lambda via cross-validation using 5-folds. Use the following Python command to generate the set of lambdas to be considered: np.linspace(0.001,100, num = 100). Notice that np is the alias for the numpy library.**
 - **Using the optimal lambda, build the ridge regression model.**

  **Finally, use this model to predict on the test dataset. Report the MSE of this model.**

In [21]:
#ridge regression
ridge_cv = RidgeCV(alphas = np.linspace(0.001, 100, num = 100), cv= 5).fit(x_train, y_train)

#extract lambda
cv_lambda = ridge_cv.alpha_
print('The optimal lambda of the ridge model is:', cv_lambda)

#building the ridge model
ridge_md = Ridge(alpha = cv_lambda).fit(x_train, y_train)

#predict on test
ridge_pred = ridge_md.predict(x_test)

#compute model mse
mse2 = np.mean(np.power(y_test-ridge_pred, 2))
print('The mse of the ridge model is:', mse2)

The optimal lambda of the ridge model is: 1.011090909090909
The mse of the ridge model is: 2714260.137126139


**(g) (8 points) Using the train dataset, build a LASSO regression model as follows:**
- **First estimate the optimal lambda via cross-validation using 5-folds. Use the following Python command to generate the set of lambdas to be considered: np.linspace(0.001, 100, num = 100). Notice that np is the alias for the numpy library.**
- **Using the optimal lambda, build the LASSO regression model**

    **Finally, use this model to predict on the test dataset. Report the MSE of this model.**

In [22]:
#lasso regression
lasso_cv = LassoCV(alphas= np.linspace(0.001,100, num=100), normalize= True, cv=5).fit(x_train, y_train)

#extract lambda
cv_lambda = lasso_cv.alpha_
print('The optimal lambda of the lasso model is:', cv_lambda)

#building the lasso model
lasso_md = Lasso(alpha = cv_lambda, normalize= True).fit(x_train, y_train)
lasso_md.coef_

#predict on test
lasso_pred = lasso_md.predict(x_test)

#compute model mse
mse3 = np.mean(np.power(y_test-lasso_pred,2 ))
print('The mse of the lasso model is:', mse3)

The optimal lambda of the lasso model is: 0.001
The mse of the lasso model is: 2708701.3146786597


**(h) (3 points) Using the results from parts (e), (f) and (g), what model would use to predict
the number of applications that a university receive?**

The mse of the linear model is: 2708734.6387247397

The mse of the ridge model is: 2714260.137126139

The mse of the lasso model is: 2708701.3146786597

I would use the lasso model because it has the lowest MSE